#KNN
##Gözlemlerin benzerliği üzerinden tahmin yapılır
##Uzaklık hesabı kullanılır

In [6]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


In [7]:
from warnings import filterwarnings
filterwarnings('ignore')

In [8]:
df=pd.read_csv("/content/Hitters.csv")
df=df.dropna()
dms=pd.get_dummies(df[['Division','League','NewLeague']])
y=df["Salary"]
X_=df.drop(['Salary','League','Division','NewLeague'],axis=1).astype('float64')
X=pd.concat([X_, dms[['League_N','Division_W','NewLeague_N']]],axis=1)
X_train, X_test, y_train, y_test =train_test_split(X,
                                                   y,
                                                   test_size=0.25,
                                                   random_state=42)

In [9]:
X_train.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
183,328.0,91.0,12.0,51.0,43.0,33.0,2.0,342.0,94.0,12.0,51.0,44.0,33.0,145.0,59.0,8.0,True,False,True
229,514.0,144.0,0.0,67.0,54.0,79.0,9.0,4739.0,1169.0,13.0,583.0,374.0,528.0,229.0,453.0,15.0,True,False,True
286,593.0,152.0,23.0,69.0,75.0,53.0,6.0,2765.0,686.0,133.0,369.0,384.0,321.0,315.0,10.0,6.0,False,True,False
102,233.0,49.0,2.0,41.0,23.0,18.0,8.0,1350.0,336.0,7.0,166.0,122.0,106.0,102.0,132.0,10.0,False,False,False
153,341.0,95.0,6.0,48.0,42.0,20.0,10.0,2964.0,808.0,81.0,379.0,428.0,221.0,158.0,4.0,5.0,True,True,True


In [10]:
knn_model = KNeighborsRegressor().fit(X_train,y_train)

In [17]:
knn_model.n_neighbors

5

In [18]:
knn_model.metric

'minkowski'

In [ ]:
dir(knn_model)

In [21]:
knn_model.predict(X_test)[0:5]

array([ 510.3334,  808.3334,  772.5   ,  125.5   , 1005.    ])

In [22]:
y_pred = knn_model.predict(X_test)

In [23]:
np.sqrt(mean_squared_error(y_test, y_pred))

426.6570764525201

##Model Tuning

In [24]:
knn_model

KNeighborsRegressor()

In [27]:
RMSE = []
for k in range(10):
  k = k+1
  knn_model=KNeighborsRegressor(n_neighbors=k).fit(X_train,y_train)
  y_pred = knn_model.predict(X_test)
  rmse=np.sqrt(mean_squared_error(y_test, y_pred))
  RMSE.append(rmse)
  print("k =",k,"icin RMSE degeri :",rmse)


k = 1 icin RMSE degeri : 455.03925390751965
k = 2 icin RMSE degeri : 415.99629571490965
k = 3 icin RMSE degeri : 420.6765370082348
k = 4 icin RMSE degeri : 428.8564674588792
k = 5 icin RMSE degeri : 426.6570764525201
k = 6 icin RMSE degeri : 423.5071669008732
k = 7 icin RMSE degeri : 414.9361222421057
k = 8 icin RMSE degeri : 413.7094731463598
k = 9 icin RMSE degeri : 417.84419990871265
k = 10 icin RMSE degeri : 421.6252180741266


In [ ]:
#GridSearchCV -> hiperparametre icin fonk.

In [28]:
knn_params = {"n_neighbors":np.arange(1,30,1)}

In [29]:
knn = KNeighborsRegressor()

In [30]:
knn_cv_model=GridSearchCV(knn, knn_params, cv=10).fit(X_train, y_train)

In [31]:
knn_cv_model.best_params_

{'n_neighbors': 8}

##Final Modeli

In [33]:
knn_tuned=KNeighborsRegressor(n_neighbors = knn_cv_model.best_params_['n_neighbors']).fit(X_train, y_train)

In [34]:
y_pred=knn_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

421.6252180741266